In [114]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
import os
import PIL
from PIL import Image
from torchvision.transforms import v2
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler


In [115]:
batch_size = 32
# Define transformations
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        self.images_files = sorted(os.listdir("crop_dataset/images"))
        self.labels_files = sorted(os.listdir("crop_dataset/labels"))

    def __len__(self):
        return len(self.images_files)
    
    def __getitem__(self, k):
        
        image_name = self.images_files[k]
        image_path = os.path.join(self.images_dir, image_name)
        #image = (read_image(image_path))
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        label_name = self.labels_files[k]
        label_path = os.path.join(self.labels_dir, label_name)
        with open(label_path, 'r') as file:
            label = int(file.readline().split()[0]) # read only first word of the label.txt file, the class

        return image, label

In [116]:
#print(len(os.listdir("crop_dataset/images"))) = 53157 (without background images)
#dataset contains 32 batch, each one of size 1662
#1662*32 = 53184
data = CustomDataset("crop_dataset/images", "crop_dataset/labels", transform=transform)
#train data and val data
train_size = int(len(data) * 0.7)
val_size = len(data)- train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size, val_size])

train_data = DataLoader(train_set, batch_size=batch_size)
val_data = DataLoader(val_set, batch_size=batch_size)

# print(len(dataset))
# print(len(train)*32)

# Iterate over the DataLoader
for batch in train_data:
    images, labels = batch
    print("Batch of images shape:", images.shape)
    print("Batch of labels:", labels)
    break

Batch of images shape: torch.Size([32, 3, 48, 48])
Batch of labels: tensor([ 2, 10,  6,  0,  1,  1,  3,  3,  3,  1,  0,  9,  4,  3,  0,  3, 10,  0,
         1,  5,  4,  6,  4,  3,  7,  3,  1,  0,  1, 11,  2,  6])


In [117]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [118]:
# Define model
class SimpleNet(nn.Module):
    # def __init__(self):
    #     super().__init__()
    #     self.flatten = nn.Flatten()
    #     self.linear_relu_stack = nn.Sequential(
    #         nn.Linear(48*48*3, 128),
    #         nn.ReLU(),
    #         nn.Linear(128, 12),
    #     )
    
    def __init__(self):
        super().__init__()
        self.mod1 = nn.Sequential(
            nn.Conv2d(3, 3, (2,2), 1),
            nn.ReLU(),
            nn.Conv2d(3, 3, (2,2), 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(6348, 12))
        
        # self.linear_relu_stack = nn.Sequential(
        #     nn.Linear(48*48*3, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 12),
        # )

    def forward(self, image):
        x = self.mod1(image)
        #x = nn.Flatten()
        return x

model = SimpleNet().to(device)
print(model)

SimpleNet(
  (mod1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=6348, out_features=12, bias=True)
  )
)


In [119]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [120]:
def train(data, model, loss_fn, optimizer):
    size = len(data.dataset)
    model.train()
    batch_count = 1
    correct = 0
    loss_sum = 0
    for batch, pairs in enumerate(data):
        images, labels = pairs[0].to(device), pairs[1].to(device)

        # Compute prediction error
        pred = model(images)
        loss = loss_fn(pred, labels)

        loss_sum += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

        #if batch_count % 800 == 0:
        # if batch_count == batch_num:
        #     loss, current = loss.item(), (batch_count) * len(images)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        batch_count += 1
    loss_sum /= batch_count
    correct /= size
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Average Loss: {loss_sum:>8f}")
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [121]:
def test(data, model, loss_fn):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    test_loss, correct = 0, 0
    batch_count = 0
    with torch.no_grad():
        for batch in data:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            # Compute prediction error
            pred = model(images)
            test_loss += loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [122]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, loss_fn, optimizer)
    test(val_data, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Training Error: 
 Accuracy: 63.1%, Average Loss: 1.223598
Test Error: 
 Accuracy: 76.0%, Avg loss: 0.979083 

Epoch 2
-------------------------------
Training Error: 
 Accuracy: 75.4%, Average Loss: 0.928545
Test Error: 
 Accuracy: 75.4%, Avg loss: 0.938233 

Epoch 3
-------------------------------
Training Error: 
 Accuracy: 78.3%, Average Loss: 0.782476
Test Error: 
 Accuracy: 78.8%, Avg loss: 0.787915 

Epoch 4
-------------------------------
Training Error: 
 Accuracy: 80.6%, Average Loss: 0.693543
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.760552 

Epoch 5
-------------------------------
Training Error: 
 Accuracy: 81.6%, Average Loss: 0.655211
Test Error: 
 Accuracy: 78.8%, Avg loss: 0.759655 

Epoch 6
-------------------------------
Training Error: 
 Accuracy: 82.4%, Average Loss: 0.625462
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.771108 

Epoch 7
-------------------------------
Training Error: 
 Accuracy: 83.3%, Average Loss: 0.600222
Te

KeyboardInterrupt: 